**BBO (Best Bid and Offer)** — это данные, которые отображают лучшую цену покупки (bid) и лучшую цену продажи (ask) на рынке в конкретный момент времени.

bbo_1000pepeusdt.csv и bbo_dogeusdt.csv
 
1. local_timestamp: Время, когда было зафиксировано состояние рынка, выраженное в локальном времени. 
2. ask_amount: Количество актива, которое доступно на продажу по лучшей (минимальной) цене продажи (ask_price).
3. ask_price: Лучшая цена продажи (цена, по которой кто-то готов продать актив).
4. bid_price: Лучшая цена покупки (цена, по которой кто-то готов купить актив).
5. bid_amount: Количество актива, которое доступно на покупку по лучшей (максимальной) цене покупки (bid_price).


**Trades (сделки)** — это данные о совершенных сделках на рынке.

trades_1000pepeusdt.csv и trades_dogeusdt.csv

1. local_timestamp: Время, когда была совершена сделка.
2. side: Направление сделки, указывающее, была ли сделка на покупку (buy) или продажу (sell).
3. price: Цена, по которой была совершена сделка.
4. amount: Количество актива, которое было куплено или продано в сделке.

По этим таблицам формируем свечи OLHC

**Свечи (OLHC candles)**

1. *Open*: цена первой сделки в окне
2. *Close*: цена последней сделки в окне
3. *Low*: минимальная цена сделки в окне
4. *High*: максимальная цена сделки в окне
5. *Средняя цена покупок*: среднее арифметическое всех цен сделок на покупку в окне
6. *Средняя цена продаж*: среднее арифметическое всех цен сделок на продажу в окне
7. *Объем покупок и продаж*: сумма объемов всех сделок на покупку и продажу в окне

In [636]:
import pandas as pd
import numpy as np

from datetime import datetime


#загрузка ордербука

bbo_1000pepeusdt=pd.read_csv('/Users/anastasiabondarenko/Downloads/md_sim-2/bbo_1000pepeusdt.csv', sep = ',')

bbo_dogeusdt=pd.read_csv('/Users/anastasiabondarenko/Downloads/md_sim-2/bbo_dogeusdt.csv', sep = ',')




#загрузка трейдов

trades_1000pepeusdt=pd.read_csv('/Users/anastasiabondarenko/Downloads/md_sim-2/trades_1000pepeusdt.csv', sep = ',')

trades_dogeusdt=pd.read_csv('/Users/anastasiabondarenko/Downloads/md_sim-2/trades_dogeusdt.csv', sep = ',')

 

#корректировка времени

bbo_1000pepeusdt['timestamp']=pd.to_datetime(bbo_1000pepeusdt['local_timestamp'], unit='us')

bbo_dogeusdt['timestamp']=pd.to_datetime(bbo_dogeusdt['local_timestamp'], unit='us')

trades_1000pepeusdt['timestamp']=pd.to_datetime(trades_1000pepeusdt['local_timestamp'], unit='us')

trades_dogeusdt['timestamp']=pd.to_datetime(trades_dogeusdt['local_timestamp'], unit='us')



#индексы -> время

bbo_1000pepeusdt.set_index('timestamp',inplace=True)

bbo_dogeusdt.set_index('timestamp',inplace=True)

trades_1000pepeusdt.set_index('timestamp',inplace=True)

trades_dogeusdt.set_index('timestamp',inplace=True)
 

T='30Min'





In [637]:
#создание OHLC свечей для pepeusdt


pepeusdt_ohlc = trades_1000pepeusdt['price'].resample(T).ohlc()

#объем покупок и продаж 

trades_1000pepeusdt['product']=trades_1000pepeusdt['price']*trades_1000pepeusdt['amount']

#cредняя цена покупок

agg_trades_1000pepeusdt_buy=trades_1000pepeusdt[trades_1000pepeusdt['side']=='buy'].groupby(pd.Grouper(freq=T))[['product','amount']].sum()

agg_trades_1000pepeusdt_buy['avg_price']=agg_trades_1000pepeusdt_buy['product']/agg_trades_1000pepeusdt_buy['amount']

agg_trades_1000pepeusdt_buy=agg_trades_1000pepeusdt_buy[['amount','avg_price']].rename(columns={'amount':'traded'}).add_suffix('_buy')

#средняя цена продаж

agg_trades_1000pepeusdt_sell=trades_1000pepeusdt[trades_1000pepeusdt['side']=='sell'].groupby(pd.Grouper(freq=T))[['product','amount']].sum()

agg_trades_1000pepeusdt_sell['avg_price']=agg_trades_1000pepeusdt_sell['product']/agg_trades_1000pepeusdt_sell['amount']

agg_trades_1000pepeusdt_sell=agg_trades_1000pepeusdt_sell[['amount','avg_price']].rename(columns={'amount':'traded'}).add_suffix('_sell')

pepeusdt_ohlc=pepeusdt_ohlc.join(agg_trades_1000pepeusdt_buy).join(agg_trades_1000pepeusdt_sell)



#создание OHLC свечей для dogeusdt


dogeusdt_ohlc = trades_dogeusdt['price'].resample(T).ohlc()

#объем покупок и продаж 

trades_dogeusdt['product']=trades_dogeusdt['price']*trades_dogeusdt['amount']

#cредняя цена покупок

agg_trades_dogeusdt_buy=trades_dogeusdt[trades_dogeusdt['side']=='buy'].groupby(pd.Grouper(freq=T))[['product','amount']].sum()

agg_trades_dogeusdt_buy['avg_price']=agg_trades_dogeusdt_buy['product']/agg_trades_dogeusdt_buy['amount']

agg_trades_dogeusdt_buy=agg_trades_dogeusdt_buy[['amount','avg_price']].rename(columns={'amount':'traded'}).add_suffix('_buy')


#средняя цена продаж


agg_trades_dogeusdt_sell=trades_dogeusdt[trades_dogeusdt['side']=='sell'].groupby(pd.Grouper(freq=T))[['product','amount']].sum()

agg_trades_dogeusdt_sell['avg_price']=agg_trades_dogeusdt_sell['product']/agg_trades_dogeusdt_sell['amount']

agg_trades_dogeusdt_sell=agg_trades_dogeusdt_sell[['amount','avg_price']].rename(columns={'amount':'traded'}).add_suffix('_sell')


dogeusdt_ohlc=dogeusdt_ohlc.join(agg_trades_dogeusdt_buy).join(agg_trades_dogeusdt_sell)

 

del bbo_1000pepeusdt,bbo_dogeusdt,trades_1000pepeusdt,trades_dogeusdt,agg_trades_dogeusdt_buy,agg_trades_dogeusdt_sell,agg_trades_1000pepeusdt_sell,agg_trades_1000pepeusdt_buy

#возвращаемся к исходным индексам
pepeusdt_ohlc = pepeusdt_ohlc.reset_index(drop= False , inplace= False)
dogeusdt_ohlc = dogeusdt_ohlc.reset_index(drop= False , inplace= False)

#сохранение свечей в файл
pepeusdt_ohlc.to_csv('/Users/anastasiabondarenko/Downloads/md_sim-2/pepeusdt_ohlc.csv')
dogeusdt_ohlc.to_csv('/Users/anastasiabondarenko/Downloads/md_sim-2/dogeusdt_ohlc.csv')

**Задание 2**

Вычисление метрик для каждой стратегии:

*1. PnL (Profit and Loss)*: финансовый результат стратегии

*2. Проторгованный объём (Traded Volume)*: суммарное количество контрактов, которые стратегия купила/продала

*3. Sharpe Ratio*: соотношение среднего дохода к стандартному отклонению доходности

*4. Sortino Ratio*: модифицированное отношение Шарпа с фокусом на негативные отклонения

*5. Максимальная просадка PnL (Max Drawdown)*: наибольшая потеря капитала

*6. Среднее время удержания позиции (Average Holding Time)*: средняя продолжительность, на которую стратегия держит актив

*7. Количество переходов через 0 (Number of Position Flips)*: сколько раз стратегия меняет направление (из покупки в продажу и наоборот)

In [639]:
#считаем среднее скользящее (МА). Если цена выше МА, то . Если цена ниже, то продаем.

#для pepeusdt

pepeusdt_ohlc['MA'] = pepeusdt_ohlc['close'].rolling(window=5).mean()

vol=pepeusdt_ohlc['close'].std()/2 #деление ну просто так (чтобы побольше точек было)

pepeusdt_ohlc['Action']=pepeusdt_ohlc.apply(lambda x: -1 if x['close']<x['MA']-vol else 1 if x['close']> x['MA']+vol else 0,axis=1).fillna(0)

pepeusdt_actions_MA = pepeusdt_ohlc[['Action']]

pepeusdt_ohlc=pepeusdt_ohlc.drop(columns=['MA', 'Action'])





#для dogeusdt

dogeusdt_ohlc['MA'] = dogeusdt_ohlc['close'].rolling(window=5).mean()

vol=dogeusdt_ohlc['close'].std()/2 #деление ну просто так (чтобы побольше точек было)

dogeusdt_ohlc['Action']=dogeusdt_ohlc.apply(lambda x: -1 if x['close']<x['MA']-vol else 1 if x['close']> x['MA']+vol else 0,axis=1).fillna(0)

dogeusdt_actions_MA=dogeusdt_ohlc[['Action']]

dogeusdt_ohlc=dogeusdt_ohlc.drop(columns=['MA','Action'])

##############################################################################################################################################

#вторая стратегия:
#считаем корреляцию на скользящем окне. Если корреляция разъезжается, то покупаем/продаем пару


corr_calculator = dogeusdt_ohlc[['close']].add_suffix('_doge').join(pepeusdt_ohlc[['close']].add_suffix('_pepe'))

corr_calculator['corr'] = corr_calculator['close_doge'].rolling(10).corr(corr_calculator['close_pepe'])

avg = corr_calculator['corr'].mean()

vol = corr_calculator['corr'].std()

corr_calculator['Action_doge'] = corr_calculator.apply(lambda x: -1 if x['corr'] < avg - vol else 1 if x['corr'] > avg + vol else 0, axis=1).fillna(0)

corr_calculator['Action_pepe'] = corr_calculator['Action_doge'].apply(lambda x: 1 if x == -1 else -1 if x == 1 else 0).fillna(0)

dogeusdt_actions_CORR = corr_calculator[['Action_doge']]
pepeusdt_actions_CORR = corr_calculator[['Action_pepe']]

dogeusdt_actions_CORR = dogeusdt_actions_CORR.rename(columns = {'Action_doge' : 'Action'})
pepeusdt_actions_CORR = pepeusdt_actions_CORR.rename(columns = {'Action_pepe' : 'Action'})
actions_pair = corr_calculator[['Action_doge', 'Action_pepe']]

del corr_calculator, avg, vol

actions_pair.to_csv('/Users/anastasiabondarenko/Downloads/md_sim-2/actions_pair.csv') 

#print(pepeusdt_ohlc.head(20))
#print(dogeusdt_ohlc.head(20))
#print(pepeusdt_actions_CORR .head(10))
#print(dogeusdt_actions_CORR .head(10))
#ТУТ ВСЕ ХОРОШО


In [640]:
#функция для симуляции торговли 

def trade_simulator(ohlc_df, actions_df, mode='close'):
    
    result = actions_df.copy()
    
    #close - стратегия для покупки/продажи по цене закрытия свечки
    #average - стратегия для покупки продажи по средней цене покупок/продаж внутри текущей свечки
    
    result['OpenPosition'] = 0  #количество открытых контрактов
    result['PositionPrice'] = 0  #цена на момент открытия позиции
    result['VWAP'] = 0  #среднее взвешенное, рассчитывается для учёта всех сделок в периоде

    
    result = result.join(ohlc_df[['close', 'low', 'high']])
    
    for r, row in result.iterrows():
        if r == 0:
            continue

        prev_row = result.iloc[r - 1]

        #режим торговли по цене закрытия свечи
        if mode == 'close':
            if row['Action'] == 0:
                #позиция не изменяется
                result.at[r, 'OpenPosition'] = prev_row['OpenPosition']
                result.at[r, 'PositionPrice'] = prev_row['PositionPrice']
                result.at[r, 'VWAP'] = prev_row['VWAP']

            elif row['Action'] == 1:
                #увеличивается открытая позиция
                result.at[r, 'OpenPosition'] = prev_row['OpenPosition'] + row['Action']
                result.at[r, 'PositionPrice'] = row['close']
                result.at[r, 'VWAP'] = prev_row['VWAP'] + row['Action'] * row['close']

            elif row['Action'] == -1:
                #уменьшается открытая позиция
                result.at[r, 'OpenPosition'] = prev_row['OpenPosition'] + row['Action']
                result.at[r, 'PositionPrice'] = row['close']
                result.at[r, 'VWAP'] = prev_row['VWAP'] + row['Action'] * row['close']

        
        #режим торговли по средней цене покупки/продажи внутри текущей свечи
        elif mode == 'average':
            if row['Action'] == 0:
                #позиция не изменяется
                result.at[r, 'OpenPosition'] = prev_row['OpenPosition']
                result.at[r, 'PositionPrice'] = prev_row['PositionPrice']
                result.at[r, 'VWAP'] = prev_row['VWAP']

            elif row['Action'] == 1:
                #увеличивается открытая позиция
                result.at[r, 'OpenPosition'] = prev_row['OpenPosition'] + row['Action']
                result.at[r, 'PositionPrice'] = (row['low'] + row['high']) / 2
                result.at[r, 'VWAP'] = prev_row['VWAP'] + row['Action'] * result.at[r, 'PositionPrice']

            elif row['Action'] == -1:
                #уменьшается открытая позиция
                result.at[r, 'OpenPosition'] = prev_row['OpenPosition'] + row['Action']
                result.at[r, 'PositionPrice'] = (row['low'] + row['high']) / 2
                result.at[r, 'VWAP'] = prev_row['VWAP'] + row['Action'] * result.at[r, 'PositionPrice']

    #заполняем пропуски предыдущими непропущенными
    result.fillna(method='ffill', inplace=True)
    
    return result

In [641]:
#СТАТИСТИКИ

def calculate_statistics(result_df):
        #рассчет PnL по стратегии по свече
        result_df['PnL'] = (result_df['OpenPosition'] * (result_df['close'] - result_df['PositionPrice']))
    
        #количество контрактов (куплено/продано)
        result_df['TradedVolume'] = result_df['OpenPosition'].abs()
    

        #общее значение прибыли/убытка
        pnl = result_df['PnL'].sum()

        #общий объем проторгованных контрактов
        traded_volume = result_df['TradedVolume'].sum()

        #максимальная просадка
        max_drawdown = (result_df['PnL'].cumsum().min() - result_df['PnL'].cumsum().max())


    
        result_df['returns'] = result_df['PnL'].pct_change().replace([np.inf, -np.inf], np.nan)

        sharpe_ratio = result_df['returns'].dropna().mean() / result_df['returns'].dropna().std()

        #учитывает только риски с убытками
        
        sortino_ratio = result_df['returns'].dropna().mean() / result_df[result_df['returns'] < 0].dropna()['returns'].std()


        #количество периодов когда есть открытая позиция 
        #holding_times = result_df.groupby(pd.Grouper(freq='D')).apply(lambda x: (x['OpenPosition'] != 0).sum())
        #average_holding_time = holding_times.mean() if not holding_times.empty else 0


    
        #переход через 0
        number_of_flips = (result_df['OpenPosition'].diff().abs() > 0).sum()

        return {
            'PnL': pnl,
            'TradedVolume': traded_volume,
            'SharpeRatio': sharpe_ratio,
            'SortinoRatio': sortino_ratio,
            'MaxDrawdown': max_drawdown,
            'NumberOfPositionFlips': number_of_flips,
        }
print(calculate_statistics(dogeusdt_result))

{'PnL': -0.2366099999999999, 'TradedVolume': 233, 'SharpeRatio': -0.11604019572918366, 'SortinoRatio': -0.08843270646130472, 'MaxDrawdown': -0.2485399999999999, 'NumberOfPositionFlips': 7}


In [642]:
#ВЕРИФИКАЦИЯ

#случайная стратегия

def random_strategy(ohlc_df):
    #-1 (продажа), 1 (покупка), 0 (ничего)
    actions = np.random.choice([-1, 0, 1], size=len(ohlc_df))
    #случайное количество контрактов для действий
    contract_size = np.random.randint(1, 10, size=len(ohlc_df))  #случайное число контрактов от 1 до 10
    #количество контрактов к действию (Action)
    ohlc_df['Action'] = actions * contract_size
    return ohlc_df[['Action']]

#стратегия с идеальным знанием будущего
def perfect_foresight_strategy(ohlc_df):
    actions = []
    for i in range(len(ohlc_df) - 1):
        if ohlc_df['close'].iloc[i + 1] > ohlc_df['close'].iloc[i]:
            actions.append(1)  #покупаем, если цена пойдет вверх
        elif ohlc_df['close'].iloc[i + 1] < ohlc_df['close'].iloc[i]:
            actions.append(-1)  #продаем, если цена пойдет вниз
        else:
            actions.append(0)  #ничего не делаем, если цена не изменилась
    actions.append(0)  # для последней свечи действие 0, т.к. нет будущей цены
    ohlc_df['Action'] = np.array(actions) 
    return ohlc_df[['Action']]


In [643]:
#данные для симуляции
random_actions = random_strategy(pepeusdt_ohlc)
perfect_actions = perfect_foresight_strategy(pepeusdt_ohlc)

#симулируем торговлю для случайной стратегии
random_simulation = trade_simulator(pepeusdt_ohlc, random_actions, mode='close')
random_stats = calculate_statistics(random_simulation)
print("Случайная стратегия:", random_stats)

#симулируем торговлю для идеальной стратегии
perfect_simulation = trade_simulator(pepeusdt_ohlc, perfect_actions, mode='close')
perfect_stats = calculate_statistics(perfect_simulation)
print("Стратегия с идеальным знанием будущего:", perfect_stats)


#стратегии среднее скользящее и корреляция на скользящем окне
#СИМУЛЯЦИЯ
pepeusdt_result = trade_simulator(pepeusdt_ohlc, pepeusdt_actions_CORR, mode = 'average')

dogeusdt_result = trade_simulator(dogeusdt_ohlc, pepeusdt_actions_MA, mode = 'close')  

pepeusdt_result.to_csv('/Users/anastasiabondarenko/Downloads/md_sim-2/pepeusdt_result.csv')
dogeusdt_result.to_csv('/Users/anastasiabondarenko/Downloads/md_sim-2/dogeusdt_result.csv')
#СТАТИСТИКИ
pepeusdt_stats = calculate_statistics(pepeusdt_result)

dogeusdt_stats = calculate_statistics(dogeusdt_result)

print("PEPEUSDT Stats:", pepeusdt_stats)

print("DOGEUSDT Stats:", dogeusdt_stats)

Случайная стратегия: {'PnL': -0.001166700000000004, 'TradedVolume': 41, 'SharpeRatio': -0.27098278974979084, 'SortinoRatio': -0.7207865639962792, 'MaxDrawdown': -0.0013333000000000017, 'NumberOfPositionFlips': 46}
Стратегия с идеальным знанием будущего: {'PnL': 0.0002351999999999979, 'TradedVolume': 3045, 'SharpeRatio': nan, 'SortinoRatio': nan, 'MaxDrawdown': -0.0002351999999999979, 'NumberOfPositionFlips': 334}
PEPEUSDT Stats: {'PnL': -0.6194517500000026, 'TradedVolume': 8703, 'SharpeRatio': 0.011563714505162682, 'SortinoRatio': 0.022907128115133967, 'MaxDrawdown': -0.6500679500000025, 'NumberOfPositionFlips': 51}
DOGEUSDT Stats: {'PnL': -0.2366099999999999, 'TradedVolume': 233, 'SharpeRatio': -0.11604019572918366, 'SortinoRatio': -0.08843270646130472, 'MaxDrawdown': -0.2485399999999999, 'NumberOfPositionFlips': 7}


/var/folders/6n/84q09_2n12930mr66_9j7ck80000gn/T/ipykernel_13317/110217927.py:34: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0.008463' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  result.at[r, 'PositionPrice'] = row['close']
/var/folders/6n/84q09_2n12930mr66_9j7ck80000gn/T/ipykernel_13317/110217927.py:35: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0.008463' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  result.at[r, 'VWAP'] = prev_row['VWAP'] + row['Action'] * row['close']
/var/folders/6n/84q09_2n12930mr66_9j7ck80000gn/T/ipykernel_13317/110217927.py:65: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  result.fillna(method='ffill', inplace=True)
/var/folders/6n/84q09_